# **Daily Swing Scanner — Pattern + Indicator Confirmations (v3.1)**

Bug fix in Lookback Audit: `np.nonzero(mask.values)` -> `np.where(mask)`.

In [10]:

import os, json, warnings, sys, requests
from typing import Dict, Any, List, Optional
import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore")

BEST_PARAMS: Dict[str, Any] = {
    "USE_RSI": True, "USE_MACD": False, "USE_ADX": False, "USE_SMA": True, "USE_BB": True,
    "RSI_LEN": 21, "RSI_LONG_MIN": 55, "RSI_SHORT_MAX": 50,
    "MACD_FAST": 12, "MACD_SLOW": 26, "MACD_SIGNAL": 9, "MACD_MODE": "hist_above0",
    "ADX_LEN": 14, "ADX_MIN": 20,
    "SMA_FAST": 10, "SMA_SLOW": 50,
    "BB_LEN": 20, "BB_STD": 2.0, "BB_SQUEEZE_PCTL": 10
}

TICKERS: List[str] = ['BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS', 'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS', 'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS', 'TVSMOTOR.NS', 'UNITDSPR.NS']


EVAL_DATE: Optional[str] = pd.Timestamp.now(tz="Asia/Kolkata").strftime("%Y-%m-%d")
# EVAL_DATE: Optional[str] = None

MIN_CONFIRMATIONS: int = 2
PATTERNS_TO_USE: List[str] = [
    "ENGULFING","PIERCING","MORNING_STAR","EVENING_STAR","HARAMI","HARAMI_CROSS",
    "HAMMER","INVERTED_HAMMER","SHOOTING_STAR","HANGING_MAN","DOJI","DARK_CLOUD_COVER",
]

LOOKBACK_DAYS: int = 600
FOLDER_DATE_MODE: str = "eval"
STRICT_EVAL_DATE: bool = False

OUT_ROOT = "outputs/scans"; os.makedirs(OUT_ROOT, exist_ok=True)

SEND_TELEGRAM: bool = False
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")
TELEGRAM_CHAT_ID   = os.getenv("TELEGRAM_CHAT_ID", "")

SEND_TELEGRAM_AUDIT: bool = False

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 200)


In [11]:

def sma(series, n): 
    return series.rolling(n, min_periods=n).mean()
def ema(series, n): 
    return series.ewm(span=n, adjust=False).mean()
def rsi(series, n=14):
    delta = series.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=series.index).rolling(n, min_periods=n).mean()
    roll_down = pd.Series(down, index=series.index).rolling(n, min_periods=n).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    return (100.0 - (100.0 / (1.0 + rs))).fillna(0.0)
def macd_line(series, fast=12, slow=26):
    return ema(series, fast) - ema(series, slow)
def macd_signal_line(macd_ln, signal=9):
    return ema(macd_ln, signal)
def true_range(df):
    prev_close = df['Close'].shift(1)
    tr = pd.concat([
        (df['High'] - df['Low']).abs(),
        (df['High'] - prev_close).abs(),
        (df['Low'] - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr
def adx(df, n=14):
    high, low, close = df['High'], df['Low'], df['Close']
    plus_dm = (high - high.shift(1)).clip(lower=0.0)
    minus_dm = (low.shift(1) - low).clip(lower=0.0)
    plus_dm[plus_dm < minus_dm] = 0.0
    minus_dm[minus_dm < plus_dm] = 0.0
    tr = true_range(df)
    atr = tr.rolling(n, min_periods=n).mean()
    plus_di = 100 * (plus_dm.rolling(n, min_periods=n).mean() / atr)
    minus_di = 100 * (minus_dm.rolling(n, min_periods=n).mean() / atr)
    dx = ( (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan) ) * 100
    adx_v = dx.rolling(n, min_periods=n).mean()
    return adx_v.fillna(0.0), plus_di.fillna(0.0), minus_di.fillna(0.0)
def bollinger_bands(series, n=20, k=2.0):
    ma = sma(series, n)
    sd = series.rolling(n, min_periods=n).std(ddof=0)
    upper = ma + k*sd
    lower = ma - k*sd
    bandwidth = (upper - lower) / ma
    return ma, upper, lower, bandwidth

def is_bullish(c,o): return c > o
def is_bearish(c,o): return c < o
def body(c,o): return (c-o).abs()
def range_(h,l): return (h-l).abs()

def pattern_flags(df, patterns):
    O,H,L,C = df['Open'], df['High'], df['Low'], df['Close']
    rng = range_(H,L)
    bod = body(C,O)
    small_body = bod <= (rng * 0.3)
    long_lower_shadow = (O - L).abs().where(C>=O, (C - L).abs()) >= rng*0.5
    long_upper_shadow = (H - O).abs().where(C>=O, (H - C).abs()) >= rng*0.5

    flags = {}
    prev = df.shift(1)

    if "ENGULFING" in patterns:
        bull = (is_bullish(C,O) & is_bearish(prev['Close'], prev['Open']) & (C >= prev['Open']) & (O <= prev['Close']))
        bear = (is_bearish(C,O) & is_bullish(prev['Close'], prev['Open']) & (C <= prev['Open']) & (O >= prev['Close']))
        flags['ENGULFING_BULL'] = bull.fillna(False)
        flags['ENGULFING_BEAR'] = bear.fillna(False)
    if "PIERCING" in patterns or "DARK_CLOUD_COVER" in patterns:
        prev_mid = (prev['Open'] + prev['Close'])/2
        piercing = is_bearish(prev['Close'], prev['Open']) & is_bullish(C,O) & (O < prev['Low']) & (C > prev_mid) & (C < prev['Open'])
        dcc = is_bullish(prev['Close'], prev['Open']) & is_bearish(C,O) & (O > prev['High']) & (C < prev_mid) & (C > prev['Close'])
        if "PIERCING" in patterns: flags['PIERCING_BULL'] = piercing.fillna(False)
        if "DARK_CLOUD_COVER" in patterns: flags['DARK_CLOUD_COVER_BEAR'] = dcc.fillna(False)
    if "MORNING_STAR" in patterns or "EVENING_STAR" in patterns:
        p2 = df.shift(2)
        ms = (is_bearish(prev['Close'], prev['Open']) & (small_body.shift(1)) & (prev['Low'] < p2['Low']) & is_bullish(C,O) & (C > p2[['Open','Close']].min(axis=1)))
        es = (is_bullish(prev['Close'], prev['Open']) & (small_body.shift(1)) & (prev['High'] > p2['High']) & is_bearish(C,O) & (C < p2[['Open','Close']].max(axis=1)))
        if "MORNING_STAR" in patterns: flags['MORNING_STAR_BULL'] = ms.fillna(False)
        if "EVENING_STAR" in patterns: flags['EVENING_STAR_BEAR'] = es.fillna(False)
    if "HARAMI" in patterns or "HARAMI_CROSS" in patterns:
        inside = (O >= prev[['Open','Close']].min(axis=1)) & (O <= prev[['Open','Close']].max(axis=1)) &                  (C >= prev[['Open','Close']].min(axis=1)) & (C <= prev[['Open','Close']].max(axis=1))
        harami_bull = is_bullish(C,O) & is_bearish(prev['Close'], prev['Open']) & inside
        harami_bear = is_bearish(C,O) & is_bullish(prev['Close'], prev['Open']) & inside
        doji = bod <= (rng * 0.1)
        cross_bull = doji & is_bearish(prev['Close'], prev['Open']) & inside
        cross_bear = doji & is_bullish(prev['Close'], prev['Open']) & inside
        if "HARAMI" in patterns:
            flags['HARAMI_BULL'] = harami_bull.fillna(False)
            flags['HARAMI_BEAR'] = harami_bear.fillna(False)
        if "HARAMI_CROSS" in patterns:
            flags['HARAMI_CROSS_BULL'] = cross_bull.fillna(False)
            flags['HARAMI_CROSS_BEAR'] = cross_bear.fillna(False)
    if "HAMMER" in patterns or "HANGING_MAN" in patterns:
        hammer_like = long_lower_shadow & (~long_upper_shadow) & (small_body)
        if "HAMMER" in patterns: flags['HAMMER_BULL'] = hammer_like.fillna(False)
        if "HANGING_MAN" in patterns: flags['HANGING_MAN_BEAR'] = hammer_like.fillna(False)
    if "INVERTED_HAMMER" in patterns or "SHOOTING_STAR" in patterns:
        inv_like = long_upper_shadow & (~long_lower_shadow) & (small_body)
        if "INVERTED_HAMMER" in patterns: flags['INVERTED_HAMMER_BULL'] = inv_like.fillna(False)
        if "SHOOTING_STAR" in patterns: flags['SHOOTING_STAR_BEAR'] = inv_like.fillna(False)
    if "DOJI" in patterns: flags['DOJI_NEUTRAL'] = (bod <= (rng * 0.05)).fillna(False)
    return pd.DataFrame(flags, index=df.index).fillna(False)


In [12]:

def add_indicators(df: pd.DataFrame, p: Dict[str, Any]) -> pd.DataFrame:
    close = df['Close']
    if p.get('USE_RSI', False):
        df['RSI'] = rsi(close, p['RSI_LEN'])
    if p.get('USE_MACD', False):
        macd_ln = macd_line(close, p['MACD_FAST'], p['MACD_SLOW'])
        df['MACD_LINE'] = macd_ln
        df['MACD_SIGNAL'] = macd_signal_line(macd_ln, p['MACD_SIGNAL'])
        df['MACD_HIST'] = df['MACD_LINE'] - df['MACD_SIGNAL']
    if p.get('USE_ADX', False):
        df['ADX'], df['+DI'], df['-DI'] = adx(df, p['ADX_LEN'])
    if p.get('USE_SMA', False):
        df['SMA_FAST'] = sma(close, p['SMA_FAST'])
        df['SMA_SLOW'] = sma(close, p['SMA_SLOW'])
    if p.get('USE_BB', False):
        bb_ma, bb_up, bb_lo, bb_bw = bollinger_bands(close, p['BB_LEN'], p['BB_STD'])
        df['BB_MA'], df['BB_UP'], df['BB_LO'], df['BB_BW'] = bb_ma, bb_up, bb_lo, bb_bw
    return df

def indicator_conditions(df: pd.DataFrame, p: Dict[str, Any], side: str) -> Dict[str, pd.Series]:
    conds = {}
    if p.get('USE_RSI', False) and 'RSI' in df:
        conds['RSI'] = (df['RSI'] >= p['RSI_LONG_MIN']) if side=='long' else (df['RSI'] <= p['RSI_SHORT_MAX'])
    if p.get('USE_MACD', False) and {'MACD_LINE','MACD_SIGNAL','MACD_HIST'}.issubset(df.columns):
        if p['MACD_MODE'] == 'hist_above0':
            conds['MACD'] = (df['MACD_HIST'] > 0.0) if side=='long' else (df['MACD_HIST'] < 0.0)
        else:
            conds['MACD'] = (df['MACD_LINE'] > df['MACD_SIGNAL']) if side=='long' else (df['MACD_LINE'] < df['MACD_SIGNAL'])
    if p.get('USE_ADX', False) and 'ADX' in df:
        conds['ADX'] = df['ADX'] >= p['ADX_MIN']
    if p.get('USE_SMA', False) and {'SMA_FAST','SMA_SLOW'}.issubset(df.columns):
        conds['SMA'] = (df['SMA_FAST'] > df['SMA_SLOW']) if side=='long' else (df['SMA_FAST'] < df['SMA_SLOW'])
    if p.get('USE_BB', False) and 'BB_BW' in df and p.get('BB_SQUEEZE_PCTL') is not None:
        thresh = df['BB_BW'].rolling(252, min_periods=252).quantile(p['BB_SQUEEZE_PCTL']/100.0)
        conds['BB'] = df['BB_BW'] <= thresh
    return conds

def indicator_confirmations(df: pd.DataFrame, p: Dict[str, Any], side: str, min_conf: int) -> pd.Series:
    conds = indicator_conditions(df, p, side)
    if not conds:
        return pd.Series(True, index=df.index)
    stacked = pd.concat(list(conds.values()), axis=1)
    ok = (stacked.sum(axis=1) >= min_conf)
    return ok.fillna(False)

def generate_entries(df: pd.DataFrame, p: Dict[str, Any], patterns: List[str], min_conf: int):
    pats = pattern_flags(df, patterns)
    bull_cols = [c for c in pats.columns if c.endswith('_BULL')]
    bear_cols = [c for c in pats.columns if c.endswith('_BEAR')]
    long_pat  = pats[bull_cols].any(axis=1) if bull_cols else pd.Series(False, index=df.index)
    short_pat = pats[bear_cols].any(axis=1) if bear_cols else pd.Series(False, index=df.index)

    long_conf  = indicator_confirmations(df, p, 'long',  min_conf)
    short_conf = indicator_confirmations(df, p, 'short', min_conf)

    long_entry  = long_pat  & long_conf
    short_entry = short_pat & short_conf
    return long_entry.fillna(False), short_entry.fillna(False), pats


In [13]:

def download_data(tickers: List[str], lookback_days: int) -> Dict[str, pd.DataFrame]:
    start = (pd.Timestamp.now(tz="Asia/Kolkata") - pd.Timedelta(days=lookback_days)).strftime("%Y-%m-%d")
    data = {}
    for t in tickers:
        df = yf.download(t, start=start, end=None, interval="1d", progress=False, auto_adjust=True, multi_level_index=False)
        if df is None or df.empty:
            continue
        data[t] = df.dropna().copy()
    return data

def choose_eval_date(data: Dict[str, pd.DataFrame], eval_date: Optional[str]) -> Optional[pd.Timestamp]:
    if eval_date:
        return pd.to_datetime(eval_date)
    last_dates = [df.index[-1].normalize() for df in data.values() if not df.empty]
    if not last_dates:
        return None
    return max(last_dates)


In [14]:

def scan_once(tickers: List[str], p: Dict[str, Any], patterns: List[str], min_conf: int, eval_date: Optional[str],
              strict_eval: bool = False):
    data = download_data(tickers, LOOKBACK_DAYS)
    if not data:
        print("No data downloaded. Check tickers/connectivity.")
        return pd.DataFrame(), None

    chosen_date = choose_eval_date(data, eval_date)
    if chosen_date is None:
        print("Could not determine evaluation date.")
        return pd.DataFrame(), None

    rows = []
    for t, df in data.items():
        df = add_indicators(df, p)
        long_entry, short_entry, pats = generate_entries(df, p, patterns, min_conf)

        mask = (df.index.normalize() == chosen_date.normalize())
        used_fallback = False
        if mask.any():
            idx = np.where(mask)[0][0]
        else:
            if strict_eval:
                continue
            last_idx = df.index[-1]
            if last_idx.normalize() <= chosen_date.normalize():
                idx = len(df) - 1
                used_fallback = True
            else:
                continue

        bar_ts = df.index[idx]
        side = None
        if long_entry.iloc[idx]: side = "long"
        elif short_entry.iloc[idx]: side = "short"
        if side is None: continue

        if side == "long":
            pats_cols = [c for c in pats.columns if c.endswith('_BULL') and pats[c].iloc[idx]]
        else:
            pats_cols = [c for c in pats.columns if c.endswith('_BEAR') and pats[c].iloc[idx]]
        pattern_names = ",".join(pats_cols) if pats_cols else ""

        enabled = [k for k in ['USE_RSI','USE_MACD','USE_ADX','USE_SMA','USE_BB'] if p.get(k, False)]
        conds = indicator_conditions(df, p, side)
        conf_true = [name for name, ser in conds.items() if bool(ser.iloc[idx])]

        snapshot = {}
        for col in ['RSI','MACD_LINE','MACD_SIGNAL','MACD_HIST','ADX','+DI','-DI','SMA_FAST','SMA_SLOW','BB_BW']:
            if col in df.columns:
                v = df[col].iloc[idx]
                if pd.notna(v):
                    snapshot[col] = float(v)

        days_diff = int((chosen_date.normalize() - bar_ts.normalize()).days)

        rows.append(dict(
            eval_date=str(chosen_date.date()),
            bar_date=str(pd.Timestamp(bar_ts).date()),
            used_fallback=bool(used_fallback),
            days_diff=days_diff,
            ticker=t,
            side=side,
            close=float(df['Close'].iloc[idx]),
            patterns=pattern_names,
            indicators_enabled=",".join(enabled),
            indicator_conf_true=",".join(conf_true),
            indicator_values=json.dumps(snapshot, ensure_ascii=False),
        ))

    return pd.DataFrame(rows), chosen_date

def folder_timestamp(chosen_date: Optional[pd.Timestamp], mode: str) -> pd.Timestamp:
    if mode == "today":
        return pd.Timestamp.now(tz="Asia/Kolkata").normalize()
    return (chosen_date.normalize() if chosen_date is not None else pd.Timestamp.now(tz="Asia/Kolkata").normalize())

def save_scan(df: pd.DataFrame, params: Dict[str, Any], chosen_date: Optional[pd.Timestamp], mode: str = "eval"):
    dts = folder_timestamp(chosen_date, mode)
    dstr = dts.strftime("%Y-%m-%d")
    out_dir = os.path.join(OUT_ROOT, dstr)
    os.makedirs(out_dir, exist_ok=True)
    scan_path = os.path.join(out_dir, "scan.csv")
    params_path = os.path.join(out_dir, "params.json")
    df.to_csv(scan_path, index=False)
    with open(params_path, "w", encoding="utf-8") as f:
        json.dump(params, f, ensure_ascii=False, indent=2)
    print(f"Saved: {scan_path}")
    print(f"Saved: {params_path}")
    return scan_path, params_path

def format_telegram_summary(df: pd.DataFrame, run_date: str) -> str:
    if df.empty:
        return f"<b>Daily Swing Scanner</b> — {run_date}\nNo signals."
    parts = [f"<b>Daily Swing Scanner</b> — {run_date}"]
    for side in ["long", "short"]:
        sdf = df[df["side"] == side]
        if sdf.empty:
            continue
        parts.append(f"\n<b>{side.upper()} ({len(sdf)})</b>")
        for _, row in sdf.sort_values(["ticker"]).iterrows():
            parts.append(
                f"• <b>{row['ticker']}</b> [{row['bar_date']}] @ {row['close']:.2f} "
                f"— patterns: {row['patterns'] or '-'}; conf: {row['indicator_conf_true'] or '-'}"
                + ("" if not row.get("used_fallback", False) else " (fallback)")
            )
    return "\n".join(parts)

def chunk_text(text: str, max_len: int = 3800) -> List[str]:
    chunks, buf, count = [], [], 0
    for line in text.splitlines():
        if count + len(line) + 1 > max_len:
            chunks.append("\n".join(buf))
            buf, count = [line], len(line) + 1
        else:
            buf.append(line); count += len(line) + 1
    if buf: chunks.append("\n".join(buf))
    return chunks

def send_telegram_message(text: str, token: str, chat_id: str, parse_mode: str = "HTML"):
    if not token or not chat_id:
        print("Telegram token/chat_id not provided; skipping send.")
        return False
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": parse_mode, "disable_web_page_preview": True}
    try:
        r = requests.post(url, data=payload, timeout=15)
        if r.status_code != 200:
            print("Telegram send error:", r.text)
            return False
        return True
    except Exception as e:
        print("Telegram send exception:", e)
        return False


In [15]:

if BEST_PARAMS.get('USE_SMA', False) and BEST_PARAMS['SMA_FAST'] >= BEST_PARAMS['SMA_SLOW']:
    raise ValueError("Invalid SMA config: SMA_FAST must be < SMA_SLOW")

signals_df, chosen_ts = scan_once(
    tickers=TICKERS,
    p=BEST_PARAMS,
    patterns=PATTERNS_TO_USE,
    min_conf=MIN_CONFIRMATIONS,
    eval_date=EVAL_DATE,
    strict_eval=STRICT_EVAL_DATE,
)

if signals_df.empty:
    print("No signals for the chosen date.")
else:
    display(signals_df.sort_values(['side','ticker','bar_date']))
    save_scan(signals_df, BEST_PARAMS, chosen_ts, mode=FOLDER_DATE_MODE)

    if SEND_TELEGRAM:
        run_date = (chosen_ts if chosen_ts is not None else pd.Timestamp.now(tz='Asia/Kolkata')).strftime('%Y-%m-%d')
        msg = format_telegram_summary(signals_df, run_date)
        for chunk in chunk_text(msg):
            ok = send_telegram_message(chunk, TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, parse_mode="HTML")
            if not ok:
                print("Failed to send one or more Telegram messages.")
                break


,eval_date,bar_date,used_fallback,days_diff,ticker,side,close,patterns,indicators_enabled,indicator_conf_true,indicator_values
0,2025-09-25,2025-09-24,True,1,BHARTIARTL.NS,long,1931.099976,INVERTED_HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 56.066175954977645, ""SMA_FAST"": 1933.0..."
1,2025-09-25,2025-09-24,True,1,CHOLAFIN.NS,long,1630.199951,ENGULFING_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 69.49367088607595, ""SMA_FAST"": 1574.36..."
2,2025-09-25,2025-09-24,True,1,DIXON.NS,long,18161.000000,"ENGULFING_BULL,MORNING_STAR_BULL","USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 79.88649378781466, ""SMA_FAST"": 18120.5..."
6,2025-09-25,2025-09-24,True,1,MAZDOCK.NS,long,2940.199951,INVERTED_HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 68.96858144194562, ""SMA_FAST"": 2939.68..."
5,2025-09-25,2025-09-24,True,1,MFSL.NS,long,1584.900024,HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","SMA,BB","{""RSI"": 43.29203397571817, ""SMA_FAST"": 1571.85..."
7,2025-09-25,2025-09-24,True,1,MUTHOOTFIN.NS,long,3088.300049,ENGULFING_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 76.73878624649583, ""SMA_FAST"": 2976.03..."
8,2025-09-25,2025-09-24,True,1,SRF.NS,long,2907.100098,HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,BB","{""RSI"": 56.40629178518464, ""SMA_FAST"": 2940.46..."
3,2025-09-25,2025-09-24,True,1,HDFCBANK.NS,short,951.049988,HANGING_MAN_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA,BB","{""RSI"": 37.78018937247592, ""SMA_FAST"": 965.175..."
4,2025-09-25,2025-09-24,True,1,ICICIBANK.NS,short,1382.699951,SHOOTING_STAR_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA,BB","{""RSI"": 36.991565636362104, ""SMA_FAST"": 1408.1..."


Saved: outputs/scans/2025-09-25/scan.csv
Saved: outputs/scans/2025-09-25/params.json


## Lookback Audit — last N business days a signal fired

In [16]:

AUDIT_LOOKBACK_DAYS: int = 30
AUDIT_MAX_ROWS_PER_TICKER: int = 20


In [17]:

def audit_recent_signals(tickers: List[str], p: Dict[str, Any], patterns: List[str], min_conf: int,
                         anchor_eval_date: Optional[str], n_days: int):
    lb = max(LOOKBACK_DAYS, n_days + 300)
    data = download_data(tickers, lb)
    if not data:
        print("No data downloaded for audit.")
        return pd.DataFrame(), None

    chosen_date = choose_eval_date(data, anchor_eval_date)
    if chosen_date is None:
        print("Could not determine audit anchor date.")
        return pd.DataFrame(), None

    days = pd.bdate_range(end=chosen_date.normalize(), periods=n_days).normalize()

    rows = []
    for t, df in data.items():
        di = add_indicators(df.copy(), p)
        long_entry, short_entry, pats = generate_entries(di, p, patterns, min_conf)

        mask = di.index.normalize().isin(days)  # ndarray of bools
        idxs = np.where(mask)[0]                # <-- FIX: use np.where(mask)

        if len(idxs) == 0:
            continue

        for idx in idxs:
            side = None
            if long_entry.iloc[idx]: side = "long"
            elif short_entry.iloc[idx]: side = "short"
            if side is None:
                continue

            if side == "long":
                pats_cols = [c for c in pats.columns if c.endswith('_BULL') and pats[c].iloc[idx]]
            else:
                pats_cols = [c for c in pats.columns if c.endswith('_BEAR') and pats[c].iloc[idx]]
            pattern_names = ",".join(pats_cols) if pats_cols else ""

            enabled = [k for k in ['USE_RSI','USE_MACD','USE_ADX','USE_SMA','USE_BB'] if p.get(k, False)]
            conds = indicator_conditions(di, p, side)
            conf_true = [name for name, ser in conds.items() if bool(ser.iloc[idx])]

            snapshot = {}
            for col in ['RSI','MACD_LINE','MACD_SIGNAL','MACD_HIST','ADX','+DI','-DI','SMA_FAST','SMA_SLOW','BB_BW']:
                if col in di.columns:
                    v = di[col].iloc[idx]
                    if pd.notna(v):
                        snapshot[col] = float(v)

            rows.append(dict(
                eval_anchor=str(chosen_date.date()),
                bar_date=str(pd.Timestamp(di.index[idx]).date()),
                ticker=t,
                side=side,
                close=float(di['Close'].iloc[idx]),
                patterns=pattern_names,
                indicators_enabled=",".join(enabled),
                indicator_conf_true=",".join(conf_true),
                indicator_values=json.dumps(snapshot, ensure_ascii=False),
            ))

    out = pd.DataFrame(rows).sort_values(["bar_date","ticker"]).reset_index(drop=True)
    return out, chosen_date

def save_audit(df: pd.DataFrame, chosen_date: Optional[pd.Timestamp], n_days: int, mode: str = "eval"):
    dts = folder_timestamp(chosen_date, mode)
    dstr = dts.strftime("%Y-%m-%d")
    out_dir = os.path.join(OUT_ROOT, dstr)
    os.makedirs(out_dir, exist_ok=True)
    path = os.path.join(out_dir, f"audit_last_{n_days}_days.csv")
    df.to_csv(path, index=False)
    print(f"Saved audit: {path}")
    return path


In [18]:

audit_df, audit_anchor = audit_recent_signals(
    tickers=TICKERS,
    p=BEST_PARAMS,
    patterns=PATTERNS_TO_USE,
    min_conf=MIN_CONFIRMATIONS,
    anchor_eval_date=EVAL_DATE,
    n_days=AUDIT_LOOKBACK_DAYS,
)

if audit_df.empty:
    print("No audit signals in the chosen window.")
else:
    _disp = audit_df.groupby('ticker').head(AUDIT_MAX_ROWS_PER_TICKER).copy()
    display(_disp)
    save_audit(audit_df, audit_anchor, AUDIT_LOOKBACK_DAYS, mode=FOLDER_DATE_MODE)


,eval_anchor,bar_date,ticker,side,close,patterns,indicators_enabled,indicator_conf_true,indicator_values
0,2025-09-25,2025-08-18,BDL.NS,short,1604.951782,SHOOTING_STAR_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 38.04213630612594, ""SMA_FAST"": 1558.63..."
1,2025-09-25,2025-08-18,BHARTIARTL.NS,short,1892.300049,HANGING_MAN_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 47.15039808140192, ""SMA_FAST"": 1889.85..."
2,2025-09-25,2025-08-18,DIVISLAB.NS,short,6164.500000,SHOOTING_STAR_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 32.83538081877451, ""SMA_FAST"": 6142.85..."
3,2025-09-25,2025-08-18,KOTAKBANK.NS,short,2001.400024,SHOOTING_STAR_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 32.8186123897453, ""SMA_FAST"": 1984.939..."
4,2025-09-25,2025-08-18,MAXHEALTH.NS,long,1211.599976,INVERTED_HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","SMA,BB","{""RSI"": 44.67022332102601, ""SMA_FAST"": 1252.99..."
...,...,...,...,...,...,...,...,...,...
279,2025-09-25,2025-09-24,ICICIBANK.NS,short,1382.699951,SHOOTING_STAR_BEAR,"USE_RSI,USE_SMA,USE_BB","RSI,SMA,BB","{""RSI"": 36.991565636362104, ""SMA_FAST"": 1408.1..."
280,2025-09-25,2025-09-24,MAZDOCK.NS,long,2940.199951,INVERTED_HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 68.96858144194562, ""SMA_FAST"": 2939.68..."
281,2025-09-25,2025-09-24,MFSL.NS,long,1584.900024,HAMMER_BULL,"USE_RSI,USE_SMA,USE_BB","SMA,BB","{""RSI"": 43.29203397571817, ""SMA_FAST"": 1571.85..."
282,2025-09-25,2025-09-24,MUTHOOTFIN.NS,long,3088.300049,ENGULFING_BULL,"USE_RSI,USE_SMA,USE_BB","RSI,SMA","{""RSI"": 76.73878624649583, ""SMA_FAST"": 2976.03..."


Saved audit: outputs/scans/2025-09-25/audit_last_30_days.csv
